<a href="https://colab.research.google.com/github/AdamClarkStandke/ReinforcementLearning/blob/main/forward_weights_90000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pathlib
import argparse
import torch
import torch.nn as nn
import ddpg

DEFAULT_INPUT_DIM = 28
ACTIONS_DIM = 4


def write_prefix(fd):
    fd.write("""from . import nn

""")


def write_weights(fd, weights):
    fd.write("WEIGHTS = [\n")
    for w, b in weights:
        fd.write("(%s, [%s]),\n" % (
            w.tolist(),  b.tolist()
        ))
    fd.write("]\n")


def write_forward_pass(fd, forward_pass):
    fd.write("""

def forward(x):
""")

    for f in forward_pass:
        fd.write("    %s\n" % f)

    fd.write("    return x\n")


def write_suffix(fd, input_dim):
    fd.write(f"""

def test():
    x = [[0.0]] * {input_dim}
    y = forward(x)
    print(y)
    
    
def show():
    for idx, (w, b) in enumerate(WEIGHTS):
        print("Layer %d:" % (idx+1))
        print("W: (%d, %d), B: (%d, %d)" % (len(w), len(w[0]), len(b), len(b[0])))

""")
    pass



output_path = pathlib.Path('move_weights.py')

act_net = ddpg.DDPGActor(DEFAULT_INPUT_DIM, ACTIONS_DIM)
act_net.load_state_dict(torch.load('best_+0.087_90000.dat', map_location=lambda storage, loc: storage))

weights_data = []
forward_pass = []

for m in act_net.net:
  if isinstance(m, nn.Linear):
    w = [m.weight.detach().numpy(), m.bias.detach().numpy()]
    forward_pass.append(f"x = nn.linear(x, WEIGHTS[{len(weights_data)}])")
    weights_data.append(w)
  elif isinstance(m, nn.ReLU):
    forward_pass.append("x = nn.relu(x)")
  elif isinstance(m, nn.Tanh):
    forward_pass.append("x = nn.tanh(x)")
  else:
    print('Unsupported layer! %s' % m)

with output_path.open("wt", encoding='utf-8') as fd_out:
  write_prefix(fd_out)
  write_weights(fd_out, weights_data)
  write_forward_pass(fd_out, forward_pass)
  write_suffix(fd_out,DEFAULT_INPUT_DIM)
